# Chapter 13: Concurrency 🚀

- Threads
- Multiprocessing
- Futures 
- AsyncIO

**By Will Norris**

## Sequential Programming: 
- Sequential programming model is intuitive and natural
    - Do things **one step at a time** (The way humans think)
- In programming languages: 
    - Each of these real-world actions is an abstraction for a sequence of finer-grained actions
    - Flow: ```Open the cupboard, select a tea, check water level in kettle, if low: add more, boil water, pour water in cup, wait for tea...``` 
- **But**, what we do while the water is boiling is up to us
    - Do we simply wait? 
    - Or, do we do other tasks such as starting our toast or fetching the newspaper (asynchronous tasks)
        - The whole time aware that we are waiting for our water to boil!
- Tea kettle makers know people tend to operate asynchronously, so they add a warning for when your tea is done, to remind you to come back to the task at hand. 
    - Finding the right balance of sequentiality and asynchrony is a characteristic of efficient people, **the same is true of efficient programs**

### Why Concurrency? 
- At some point it is not cost efficient to buy a faster machine (**scaling vertically**)
- Instead of scaling computation up, we can go out (**scaling horizontally**)
    - Allows us to use cheap hardware, and accomplish pieces of computation across a set of threads/processors/nodes 
- In modern computing, we can divide the problem entirely across nodes (processors) 
    - In legacy computing, we could take advantage of "switching", which means rapidly swapping between threads on a single process to accompish multiple things "at once" (time sharing systems) 
    

## Multiple Processes: 
__Motivating Factors:__
- **Resource Utilizaton:** 
    - Programs are always waiting for external operations (File I/O), and can't do anything while they wait. Let's use that time!
- **Fairness:**
    - Multiple users and programs may have equal claim on the machine's resources. We want to let them share "slices" of time rather than give one before the other 
- **Convenience:**
    - It is easier to write several programs that each perform a single task and have them coordinate with each other when needed than to write one big program. 

## Threads: 
- Threads allow multiple streams of program control flow to coexist within a process. 
- They share process-wide resources (memory, file handles) 
    - But, each thread has its own program counter, stack, and local variables 
- Threads provide a natural decomposition for exploiting hardware parallelism when we have multiple processors
    -  multiple threads within the same program can be scheduled simultaneously on multi CPU's
- Most modern OS's treat threads as **lightweight processses** and use them (not processes) as the basic unit of scheduling

![](https://imgur.com/5mte34P.png)

In [17]:
from threading import Thread

class InputReader(Thread):
    def run(self):
        self.line_of_text = input()

In [19]:
print("Enter any text and press enter: ")
thread = InputReader()
thread.start()

count = result = 1
while thread.is_alive():
    result = count * count 
    count += 1

print("calculated squares up to {0} * {0} = {1}".format(
    count, result))
print("while you typed '{}'".format(thread.line_of_text))

Enter any text and press enter: 
will is cool
calculated squares up to 4552405 * 4552405 = 20724382179216
while you typed 'will is cool'


In [15]:
import json 
from urllib.request import urlopen 
import time 

CITIES = ['Edmonton', 'Victoria', 'Winnipeg', 'Fredericton',
       "St. John's", 'Halifax', 'Toronto', 'Charlottetown',
          'Quebec City', 'Regina']

class TempGetter(Thread):
    def __init__(self, city):
        super().__init__()
        self.city = city
    def run(self):
        url_template = (
           'http://api.openweathermap.org/data/2.5/'
           'weather?q={},CA&units=metric')
        response = urlopen(url_template.format(self.city))
        data = json.loads(response.read().decode())
        self.temperature = data['main']['temp']
        
threads = [TempGetter(c) for c in CITIES]
start = time.time()
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()
for thread in threads:
    print("it is {0.temperature:.0f}°C in {0.city}".format(thread))
print(
   "Got {} temps in {} seconds".format(
   len(threads), time.time() - start))


Exception in thread Thread-59:
Traceback (most recent call last):
  File "/Users/williamnorris/anaconda3/envs/swepy_env/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "<ipython-input-15-a9182ff96420>", line 17, in run
    response = urlopen(url_template.format(self.city))
  File "/Users/williamnorris/anaconda3/envs/swepy_env/lib/python3.6/urllib/request.py", line 223, in urlopen
    return opener.open(url, data, timeout)
  File "/Users/williamnorris/anaconda3/envs/swepy_env/lib/python3.6/urllib/request.py", line 532, in open
    response = meth(req, response)
  File "/Users/williamnorris/anaconda3/envs/swepy_env/lib/python3.6/urllib/request.py", line 642, in http_response
    'http', request, response, code, msg, hdrs)
  File "/Users/williamnorris/anaconda3/envs/swepy_env/lib/python3.6/urllib/request.py", line 570, in error
    return self._call_chain(*args)
  File "/Users/williamnorris/anaconda3/envs/swepy_env/lib/python3.6/urllib/request.py", line 50

Exception in thread Thread-63:
Traceback (most recent call last):
  File "/Users/williamnorris/anaconda3/envs/swepy_env/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "<ipython-input-15-a9182ff96420>", line 17, in run
    response = urlopen(url_template.format(self.city))
  File "/Users/williamnorris/anaconda3/envs/swepy_env/lib/python3.6/urllib/request.py", line 223, in urlopen
    return opener.open(url, data, timeout)
  File "/Users/williamnorris/anaconda3/envs/swepy_env/lib/python3.6/urllib/request.py", line 532, in open
    response = meth(req, response)
  File "/Users/williamnorris/anaconda3/envs/swepy_env/lib/python3.6/urllib/request.py", line 642, in http_response
    'http', request, response, code, msg, hdrs)
  File "/Users/williamnorris/anaconda3/envs/swepy_env/lib/python3.6/urllib/request.py", line 570, in error
    return self._call_chain(*args)
  File "/Users/williamnorris/anaconda3/envs/swepy_env/lib/python3.6/urllib/request.py", line 50

AttributeError: 'TempGetter' object has no attribute 'temperature'